In [ ]:
%pip uninstall -y qiskit-terra
%pip uninstall -y qiskit
%pip install --upgrade qiskit
%pip install --upgrade qiskit-optimization
%pip install --upgrade qiskit-algorithms
%pip install --upgrade qiskit-visualization
%pip install --upgrade pyqubo
%pip install --upgrade numpy as np
%pip install --upgrade 

In [ ]:
import numpy as np
from pyqubo import Array
from typing import List, Tuple
from collections import defaultdict

from qiskit_algorithms.utils import algorithm_globals
from qiskit_optimization import QuadraticProgram
from qiskit_algorithms import QAOA, NumPyMinimumEigensolver
from qiskit_algorithms.optimizers import COBYLA
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit.primitives import BackendSampler
from qiskit_braket_provider import BraketProvider

In [ ]:
from qiskit.visualization import plot_state_qsphere

In [20]:
sampler = BackendSampler(
    backend=BraketProvider().get_backend("SV1"),
    options={"shots": 3},
)
# プレイヤー名テーブル
players=["PlayerA","PlayerB","PlayerC","PlayerD","PlayerE","PlayerF"]

# マッチングに使用するパラメータを定義
# 実際の数値は桁や単位がバラバラなことが多いため、標準化や重みづけを行う必要がある。
# ダメージテーブル
damage_rate = [1, 2, 3, 4, 5, 6]
# 射程
effective_range = [1, 2, 3, 4, 5, 6]
# 速度
speed = [10, 10, 10, 10, 10, 10]
# レート
rating = [10, 9, 3, 2, 8, 5]

#プレイヤー数
N = len(players)
#チームの数
K = 3

# ペナルティ係数定義
lam = 500
x = Array.create(name='x', shape=(N,K), vartype='BINARY')

# コスト関数定義
damage_rate_cost = 1/K * sum((sum(damage_rate[i]*x[i,k] for i in range(N)) - 1/K * sum(sum(damage_rate[i]*x[i,k] for i in range(N)) for k in range(K)))**2 for k in range(K))
effective_range_cost = 1/K * sum((sum(effective_range[i]*x[i,k] for i in range(N)) - 1/K * sum(sum(effective_range[i]*x[i,k] for i in range(N)) for k in range(K)))**2 for k in range(K))
speed_cost = 1/K * sum((sum(speed[i]*x[i,k] for i in range(N)) - 1/K * sum(sum(speed[i]*x[i,k] for i in range(N)) for k in range(K)))**2 for k in range(K))
rating_cost = 1/K * sum((sum(rating[i]*x[i,k] for i in range(N)) - 1/K * sum(sum(rating[i]*x[i,k] for i in range(N)) for k in range(K)))**2 for k in range(K))

# ペナルティ関数定義
penalty = lam * sum((sum(x[i,k] for k in range(K)) -1 )**2 for i in range(N))

# qubo作成
y = damage_rate_cost + effective_range_cost + speed_cost + rating_cost + penalty
model = y.compile()
quadratic, offset = model.to_qubo()

# 量子ビット定義
qubo = QuadraticProgram()
for i in range(N):
    for j in range(K):
        qubo.binary_var("x[{}][{}]".format(i, j))

# qiskit solve
qubo.minimize(quadratic=quadratic)
algorithm_globals.random_seed = 10598
qaoa_mes = QAOA(sampler=sampler, optimizer=COBYLA(), initial_point=[0.0, 0.0])
exact_mes = NumPyMinimumEigensolver()
qaoa = MinimumEigenOptimizer(qaoa_mes)  # using QAOA
exact = MinimumEigenOptimizer(
    exact_mes
)  # using the exact classical numpy minimum eigen solver
exact_result = exact.solve(qubo)
result = exact_result.x

# 各チームのメンバーのリストを作成
players_in_team = defaultdict(list)
for i, player in enumerate(result):
    team_name = f"team_{i % K + 1}"
    if player == 1.0:
        player=players[int(i/K)]
        players_in_team[team_name].append(player)

# 結果を出力
for team_name, players in players_in_team.items():
    print(f"{team_name}: {players}")

[1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0.]
team_1: ['PlayerA', 'PlayerF']
team_3: ['PlayerB', 'PlayerD']
team_2: ['PlayerC', 'PlayerE']
